In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
ratings = pd.read_csv('ml-ratings.csv')

In [3]:
preds = pd.read_csv('ml-preds.csv')

In [4]:
ratings = ratings.drop(columns='Unnamed: 0')

In [5]:
preds = preds.drop(columns='Unnamed: 0')

# 1. 전처리

In [6]:
ratings_id = ratings.userId.unique()
pred_id = preds.userId.unique()

In [7]:
ratings_id

array([  1,   2,   3,   5,   6,   7,   8,  10,  11,  12,  13,  14,  15,
        16,  18,  20,  21,  22,  23,  24,  25,  26,  28,  37,  38,  41,
        42,  43,  44,  48,  49,  52,  54,  56,  57,  58,  59,  60,  62,
        63,  64,  65,  69,  70,  72,  73,  75,  76,  77,  79,  81,  82,
        83,  84,  85,  87,  89,  90,  91,  92,  94,  95,  96,  97,  99,
       100, 101, 102, 104, 106, 109, 110, 112, 113, 114, 115, 116, 117,
       118, 119, 121, 122, 123, 125, 126, 128, 130, 135, 137, 138, 141,
       144, 145, 148, 150, 151, 152, 154, 157, 158, 159, 160, 161, 162,
       164, 167, 168, 174, 176, 177, 178, 180, 181, 183, 184, 185, 186,
       187, 188, 189, 190, 193, 194, 195, 197, 198, 199, 200, 201, 203,
       204, 206, 207, 208, 210, 213, 214, 215, 216, 217, 218, 221, 222,
       223, 224, 227, 230, 232, 233, 234, 235, 236, 237, 239, 243, 244,
       246, 248, 249, 250, 251, 253, 254, 255, 256, 257, 262, 263, 264,
       267, 268, 269, 270, 271, 272, 274, 275, 276, 277, 279, 28

In [8]:
result = ratings.merge(preds, how='left', on=['userId', 'itemId'])

In [9]:
result['rank_x'] = result.groupby('userId')['rating_x'].rank(ascending = False, method = 'first')
result['rank_y'] = result.groupby('userId')['rating_y'].rank(ascending = False, method = 'first')

# 2. MAP

In [10]:
result_userId = result.userId.unique()

In [11]:
tmp_ap_list = []
for user in result_userId:
    tmp_10 = result[result.userId == int(user)].sort_values(by='rank_x', ascending=True)[:10]
    tmp_ap = (tmp_10.rank_y <= 10).sum()
    tmp_ap_list += [tmp_ap/10]

In [12]:
def user_preccision(data10):
    tmp_prec_list = []
    for i in range(len(data10)):
        tmp_prec = (i+1)/data10.index[i]
        tmp_prec_list += [tmp_prec]
    return np.array(tmp_prec_list).sum()/len(tmp_prec_list)

In [13]:
tmp_ap_list = []
for user in result_userId:
    tmp_10 = result[result.userId == int(user)].sort_values(by='rank_x', ascending=True)[:10]
    tmp_10.index = range(1, 11)
    tmp_10_true = tmp_10[tmp_10.rank_y <=10]
    tmp_calc = user_preccision(tmp_10_true)
    if  math.isnan(tmp_calc) == True:
        tmp_ap_list += [0]
    else:
        tmp_ap_list += [tmp_calc]

C:\Users\fried_egg_\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in double_scalars
  


In [14]:
### MAP 계산
np.array(tmp_ap_list).mean()

0.5856838408480329

# 3. nDCG

In [15]:
to2 = np.array([1,1])
from3_to10 =np.log2(np.array(range(3,11)))
dcg_calc_col = np.concatenate((to2, from3_to10), axis=None)
tmp_ndcg_list = []

for user in result_userId:
    tmp_10 = result[result.userId == int(user)].sort_values(by='rank_x', ascending=True)[:10]
    tmp_dcg = tmp_10.rating_y
    tmp_perfect_dcg = tmp_10.rating_y.sort_values(ascending=False)
    ndcg_user = (tmp_dcg/dcg_calc_col).sum() / (tmp_perfect_dcg/dcg_calc_col).sum()
    tmp_ndcg_list += [ndcg_user]

In [16]:
## nDCG의 mean
np.array(tmp_ndcg_list).mean()

0.972169985026755

# 4. binary nDCG

In [17]:
to2 = np.array([1,1])
from3_to10 =np.log2(np.array(range(3,11)))
dcg_calc_col = np.concatenate((to2, from3_to10), axis=None)
tmp_ndcg_list = []

for user in result_userId:
    tmp_10 = result[result.userId == int(user)].sort_values(by='rank_x', ascending=True)[:10]
    tmp_dcg = tmp_10.rating_y >= 4
    tmp_perfect_dcg = tmp_10.rating_y.sort_values(ascending=False) >= 4
    ndcg_user = (tmp_dcg/dcg_calc_col).sum() / (tmp_perfect_dcg/dcg_calc_col).sum()
    if  math.isnan(ndcg_user) == True:
        tmp_ndcg_list += [0]
    else:
        tmp_ndcg_list += [ndcg_user]

C:\Users\fried_egg_\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.


In [18]:
### binary nDCG 계산
np.array(tmp_ndcg_list).mean()

0.7086122144564958